## Imports

In [1]:
import json, sys, regex
import torch
import GPUtil
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
##----------------------------------------------------
from keras.models import load_model
from transformers import *
from transformers import XLMRobertaConfig
from transformers import XLMRobertaModel
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, XLMRobertaModel
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel
import pickle
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset
from preprocessing import preprocess,preprocess_sentence

c:\Users\ahmed\anaconda3\lib\site-packages\transformers\generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
c:\Users\ahmed\anaconda3\lib\site-packages\transformers\generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Loading the data

In [51]:
# loading tokenizer for LSTM and GRU
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [52]:
# Load the MARBERT tokenizer
MARBERT_tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/MARBERT")

loading configuration file config.json from cache at C:\Users\ahmed/.cache\huggingface\hub\models--UBC-NLP--MARBERT\snapshots\88e1fa192dd723cf0b3563500aec46209762eb22\config.json
Model config BertConfig {
  "_name_or_path": "UBC-NLP/MARBERT",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.29.1",
  "type_vocab_size": 2,
  "use_cache": tr

In [53]:
df_test = preprocess("2018-E-c-Ar-dev.xlsx")

In [54]:
# Preprocess your dataset
x_test = df_test["Tweet"].tolist()
y_test_pre = df_test.drop(["ID","Tweet"],axis=1)
y_test = np.asarray(y_test_pre.values.tolist())
xtest_sequences = tokenizer.texts_to_sequences(x_test)
word_index = tokenizer.word_index

## MARBERT

In [55]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [56]:
# Load the MARBERT model for multi-label classification
MARBERT_model = AutoModelForSequenceClassification.from_pretrained("UBC-NLP/MARBERT", num_labels=11)

loading configuration file config.json from cache at C:\Users\ahmed/.cache\huggingface\hub\models--UBC-NLP--MARBERT\snapshots\88e1fa192dd723cf0b3563500aec46209762eb22\config.json
Model config BertConfig {
  "_name_or_path": "UBC-NLP/MARBERT",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
   

In [ ]:
# Load the MARBERT model for multi-label classification
MARBERT_model = AutoModelForSequenceClassification.from_pretrained("UBC-NLP/MARBERT", num_labels=11)

loading configuration file config.json from cache at C:\Users\ahmed/.cache\huggingface\hub\models--UBC-NLP--MARBERT\snapshots\88e1fa192dd723cf0b3563500aec46209762eb22\config.json
Model config BertConfig {
  "_name_or_path": "UBC-NLP/MARBERT",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
   

In [131]:
# Load saved model
MARBERT_model.load_state_dict(torch.load("marbert_4epochs.pt"))

<All keys matched successfully>

In [144]:
def marbert_predict(sentence, probs=False):
    encoded_input = MARBERT_tokenizer.encode_plus(
        sentence,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    # Move the input tensors to the device
    input_ids = encoded_input["input_ids"].to(device)
    attention_mask = encoded_input["attention_mask"].to(device)

    # Set the model to evaluation mode
    MARBERT_model.eval()

    # Move the model to the device
    MARBERT_model.to(device)

    # Disable gradient calculation
    with torch.no_grad():
        # Forward pass
        outputs = MARBERT_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # Convert logits to probabilities
    probabilities = torch.sigmoid(logits)
    predictions = torch.round(probabilities)
    probabilities = np.asarray(probabilities).astype('int32')[0].tolist()
    predictions = np.asarray(predictions).astype('int32')[0].tolist()
    if probs:
        return probabilities
    else:
        return predictions

In [146]:
# Lists to store predicted and true labels
all_predictions_mar = []

for sentence in x_test:
    all_predictions_mar.append(marbert_predict(sentence))


trueN_mar = 0
trueP_mar = 0
falseN_mar = 0
falseP_mar = 0
# Calculate evaluation metrics
for i in range(len(all_predictions_mar)):
    for j in range(11):
        if all_predictions_mar[i][j] == 0:
            if y_test[i][j] == 0:
                trueN_mar+=1
            if y_test[i][j] == 1:
                falseN_mar+=1
        if all_predictions_mar[i][j] == 1:
            if y_test[i][j] == 1:
                trueP_mar+=1
            if y_test[i][j] == 0:
                falseP_mar+=1

positive_mar = trueP_mar + falseN_mar
negative_mar = trueN_mar + falseP_mar

accuracy_mar = (trueN_mar + trueP_mar) / (positive_mar + negative_mar)
f1score_mar = (2 * trueP_mar) / (2*trueP_mar + falseP_mar + falseN_mar)
precision_mar = trueP_mar / (trueP_mar + falseP_mar)
recall_mar = trueP_mar / (trueP_mar + falseN_mar)

print(f"Accuracy: {accuracy_mar:.4f}")
print(f"Precision: {precision_mar:.4f}")
print(f"Recall: {recall_mar:.4f}")
print(f"F1-Score: {f1score_mar:.4f}")

Accuracy: 0.8569
Precision: 0.7191
Recall: 0.5505
F1-Score: 0.6236


## LSTM

In [85]:
lstm_model = load_model("LSTM_Model.h5")

In [76]:
EMBEDDING_DIMENSION = 100
VOCABULARY_SIZE = 22000
MAX_LENGTH = 26
OOV_TOK = '<OOV>'
TRUNCATE_TYPE = 'post'
PADDING_TYPE = 'post'

In [77]:
xtest_sequences = tokenizer.texts_to_sequences(x_test)
word_index = tokenizer.word_index

In [78]:
xtest_pad = pad_sequences(xtest_sequences, maxlen=MAX_LENGTH, padding=PADDING_TYPE, truncating=TRUNCATE_TYPE)

In [140]:
def lstm_predict(sentence,probs=False):
    pred_temp = x_test.copy()
    pred_temp[0] = sentence
    sequences = tokenizer.texts_to_sequences(pred_temp)
    pred_pad = pad_sequences(sequences,maxlen=MAX_LENGTH, padding=PADDING_TYPE, truncating=TRUNCATE_TYPE)
    pred = lstm_model.predict(pred_pad,verbose=None)
    prob = pred[0]
    if probs:
        return prob
    else:
        return np.round(prob).astype("int32").tolist()

In [143]:
all_predictions_lstm = []

pred = lstm_model.predict(xtest_pad,verbose=None)
for p in pred:
    all_predictions_lstm.append(np.round(p).astype("int32").tolist())


trueP = 0
falseP = 0
trueN = 0
falseN = 0

for i in range(len(all_predictions_lstm)):
    for j in range(11):
        if all_predictions_lstm[i][j] == 0:
            if y_test[i][j] == 0:
                trueN+=1
            if y_test[i][j] == 1:
                falseN+=1
        if all_predictions_lstm[i][j] == 1:
            if y_test[i][j] == 1:
                trueP+=1
            if y_test[i][j] == 0:
                falseP+=1

positive = trueP + falseN
negative = trueN + falseP

accuracy = (trueN + trueP) / (positive + negative)
f1score = (2 * trueP) / (2*trueP + falseP + falseN)
precision = trueP / (trueP + falseP)
recall = trueP / (trueP + falseN)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1score:.4f}")


Accuracy: 0.8365
Precision: 0.6564
Recall: 0.5058
F1-Score: 0.5713


## GRU

In [82]:
gru_model = load_model("GRU_Model_Final.h5")

In [141]:
def gru_predict(sentence,probs=False):
    pred_temp = x_test.copy()
    pred_temp[0] = sentence
    sequences = tokenizer.texts_to_sequences(pred_temp)
    pred_pad = pad_sequences(sequences,maxlen=MAX_LENGTH, padding=PADDING_TYPE, truncating=TRUNCATE_TYPE)
    pred = gru_model.predict(pred_pad,verbose=None)
    prob = pred[0]
    if probs:
        return prob
    else:
        return np.round(prob).astype("int32").tolist()

In [142]:
all_predictions_gru = []

pred = gru_model.predict(xtest_pad,verbose=None)
for p in pred:
    all_predictions_gru.append(np.round(p).astype("int32"))


trueP_gru = 0
falseP_gru = 0
trueN_gru = 0
falseN_gru = 0

for i in range(len(all_predictions_gru)):
    for j in range(11):
        if all_predictions_gru[i][j] == 0:
            if y_test[i][j] == 0:
                trueN_gru+=1
            if y_test[i][j] == 1:
                falseN_gru+=1
        if all_predictions_gru[i][j] == 1:
            if y_test[i][j] == 1:
                trueP_gru+=1
            if y_test[i][j] == 0:
                falseP_gru+=1

positive_gru = trueP_gru + falseN_gru
negative_gru = trueN_gru + falseP_gru

accuracy_gru = (trueN_gru + trueP_gru) / (positive_gru + negative_gru)
f1score_gru = (2 * trueP_gru) / (2*trueP_gru + falseP_gru + falseN_gru)
precision_gru = trueP_gru / (trueP_gru + falseP_gru)
recall_gru = trueP_gru / (trueP_gru + falseN_gru)

print(f"Accuracy: {accuracy_gru:.4f}")
print(f"Precision: {precision_gru:.4f}")
print(f"Recall: {recall_gru:.4f}")
print(f"F1-Score: {f1score_gru:.4f}")


Accuracy: 0.8246
Precision: 0.7849
Recall: 0.2554
F1-Score: 0.3854


## Ensemble

We define an ensemble layer as a weighted average layer of all probabilities from the 3 models. <br>
We select weights in relation to F1 score: <br>
    - Marbert: F1 score = 0.6236<br>
    - LSTM:    F1 score = 0.5713<br>
    - GRU:     F1 score = 0.3854<br>

We will divide each F1 score by the sum of the f1 scores to get a weight for the model, with weights adding up to 1.


In [137]:
sum_f1 = f1score + f1score_gru + f1score_mar
sum_f1

1.580343899743148

In [179]:
# Model weights:
lstm_weight = f1score / sum_f1
gru_weight = f1score_gru / sum_f1
marbert_weight = f1score_mar / sum_f1


print("LSTM: ",lstm_weight)
print("GRU: ",gru_weight)
print("MARBERT: ",marbert_weight)
print("Sum: ",gru_weight+lstm_weight+marbert_weight)

LSTM:  0.3615112783566524
GRU:  0.24387792824845264
MARBERT:  0.39461079339489485
Sum:  1.0


In [149]:
def ensemble_predict(sentence,probs=False):
    lstm_prob = lstm_predict(sentence,probs=True)
    gru_prob = gru_predict(sentence,probs=True)
    mar_prob = marbert_predict(sentence,probs=True)
    lstm_prob = [i*lstm_weight for i in lstm_prob]
    gru_prob = [i*gru_weight for i in gru_prob]
    mar_prob = [i*marbert_weight for i in mar_prob]
    probabilities = []
    for i in range(len(lstm_prob)):
        probabilities.append(lstm_prob[i] + gru_prob[i] + mar_prob[i])
    if probs:
        return probabilities
    else:
        return np.round(probabilities).astype("int32").tolist()


In [254]:
mar_preds = []
mar_probs = []
gru_preds = []
lstm_preds = []

for sentence in x_test:
    mar_preds.append(marbert_predict(sentence))
    mar_probs.append(marbert_predict(sentence,probs=True))

lstm_probs = lstm_model.predict(xtest_pad,verbose = None)
for p in lstm_probs:
    lstm_preds.append(np.round(p).astype("int32"))
gru_probs = gru_model.predict(xtest_pad,verbose = None)
for p in gru_probs:
    gru_preds.append(np.round(p).astype("int32"))


In [262]:
lstm_weight = 0.23
gru_weight = 0.32
marbert_weight = 0.51
ens_preds = np.zeros(np.asarray(mar_preds).shape)

for i in range(len(mar_preds)):
    for j in range(11):
        ens_preds[i][j] = np.round(mar_preds[i][j]*marbert_weight + lstm_preds[i][j]*lstm_weight + gru_preds[i][j]*gru_weight).astype("int32")

In [243]:
# ens_pred = np.zeros(np.asarray(all_predictions_mar2).shape)
# for i in range(len(all_predictions_mar2)):
#     for j in range(11):
#         ens_pred[i][j] = np.round(ens_probs[i][j]).astype("int32")

In [263]:
trueN_ens = 0
trueP_ens = 0
falseN_ens = 0
falseP_ens = 0
# Calculate evaluation metrics
for i in range(len(ens_preds)):
    for j in range(11):
        if ens_preds[i][j] == 0:
            if y_test[i][j] == 0:
                trueN_ens+=1
            if y_test[i][j] == 1:
                falseN_ens+=1
        if ens_preds[i][j] == 1:
            if y_test[i][j] == 1:
                trueP_ens+=1
            if y_test[i][j] == 0:
                falseP_ens+=1

positive_ens = trueP_ens + falseN_ens
negative_ens = trueN_ens + falseP_ens

accuracy_ens = (trueN_ens + trueP_ens) / (positive_ens + negative_ens)
f1score_ens = (2 * trueP_ens) / (2*trueP_ens + falseP_ens + falseN_ens)
precision_ens = trueP_ens / (trueP_ens + falseP_ens)
recall_ens = trueP_ens / (trueP_ens + falseN_ens)

print(f"Accuracy: {accuracy_ens:.4f}")
print(f"Precision: {precision_ens:.4f}")
print(f"Recall: {recall_ens:.4f}")
print(f"F1-Score: {f1score_ens:.4f}")

Accuracy: 0.8567
Precision: 0.7028
Recall: 0.5801
F1-Score: 0.6356
